# **Machine Learning Group Task 1: Feature Selection & Regression using ANN**

## Group members
Harry Karani

Stephanie Mboya

Anthony Angatia

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
# To get the L1 embeddings
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
# importing the models
from keras.models import Sequential
from keras.layers import Dense
# To normalize the data
from sklearn.preprocessing import PowerTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICSY4S1-ML-Notebooks/Housing Pricing.csv')

In [ ]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
data.shape

(1460, 77)

**Removing useless columns and Null values**

In [ ]:
data.drop('Id',axis=1,inplace=True)

In [ ]:
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [ ]:
features_with_missing=list(data.columns[data.isna().any()])
features_with_missing

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [ ]:
# drop columns with 50% missing values and above
data.dropna(axis=1,thresh=0.5*(len(data)),inplace=True)
data.shape

(1460, 76)

In [ ]:
f_fill = features_with_missing

In [ ]:
# fill missing values with forward fill
for feature in f_fill:
  data[feature].fillna(method='ffill', inplace=True)

**Encoding**

In [ ]:
original_features=list(data.columns)
len(original_features)

76

In [ ]:
#select all non-numerical data
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [ ]:
cat_features=list(cat_data.columns)
cat_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
#get numerical, nominal and ordinal features
numerical_features = list(set(original_features) - set(cat_features))
nominal_features = ['MSZoning','LandContour','LotConfig','Neighborhood']
ordinal_features = list(set(cat_features) - set(nominal_features))
target = ['SalePrice']

In [ ]:
for feature in ordinal_features:
  data[feature]=(data[feature].astype('category')).cat.codes
df_ordinal_features = data[ordinal_features]

In [ ]:
df_nominal_features=pd.get_dummies(data[nominal_features])

In [ ]:
df_numerical_features=data[numerical_features]

In [ ]:
data_encoded=pd.concat([df_numerical_features,df_nominal_features,df_ordinal_features],axis=1)

In [ ]:
data_encoded.head()

,EnclosedPorch,3SsnPorch,HalfBath,BsmtFinSF1,MasVnrArea,BsmtHalfBath,MoSold,2ndFlrSF,GarageCars,LotFrontage,SalePrice,YearBuilt,WoodDeckSF,BedroomAbvGr,OverallQual,OpenPorchSF,1stFlrSF,MSSubClass,TotalBsmtSF,TotRmsAbvGrd,PoolArea,MiscVal,YearRemodAdd,GrLivArea,KitchenAbvGr,GarageArea,Fireplaces,LotArea,YrSold,BsmtFullBath,FullBath,ScreenPorch,BsmtUnfSF,OverallCond,BsmtFinSF2,GarageYrBlt,LowQualFinSF,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Exterior1st,LotShape,Functional,LandSlope,FireplaceQu,SaleCondition,SaleType,Foundation,Electrical,BsmtFinType2,Condition2,BsmtQual,Heating,HouseStyle,Exterior2nd,ExterQual,Utilities,Street,GarageCond,RoofStyle,ExterCond,GarageQual,KitchenQual,BldgType,GarageType,RoofMatl,PavedDrive,GarageFinish,BsmtExposure,CentralAir,BsmtFinType1,Condition1,HeatingQC,MasVnrType,BsmtCond
0,0,0,1,706,196.0,0,2,854,2,65.0,208500,2003,0,3,7,61,856,60,856,8,0,0,2003,1710,1,548,0,8450,2008,1,2,0,150,5,0,2003.0,0,0,0,0,...,0,0,0,0,0,12,3,6,0,-1,4,8,2,4,5,2,2,1,5,13,2,0,1,4,1,4,4,2,0,1,1,2,1,3,1,2,2,0,1,3
1,0,0,0,978,0.0,1,5,0,2,80.0,181500,1976,298,3,6,0,1262,20,1262,6,0,0,1976,1262,1,460,1,9600,2007,0,2,0,284,8,0,1976.0,0,0,0,0,...,0,0,0,0,1,8,3,6,0,4,4,8,1,4,5,2,2,1,2,8,3,0,1,4,1,4,4,3,0,1,1,2,1,1,1,0,1,0,2,3
2,0,0,1,486,162.0,0,9,866,2,68.0,223500,2001,0,3,7,42,920,60,920,6,0,0,2002,1786,1,608,1,11250,2008,1,2,0,434,5,0,2001.0,0,0,0,0,...,0,0,0,0,0,12,0,6,0,4,4,8,2,4,5,2,2,1,5,13,2,0,1,4,1,4,4,2,0,1,1,2,1,2,1,2,2,0,1,3
3,272,0,0,216,0.0,0,2,756,3,60.0,140000,1915,0,3,7,35,961,70,756,7,0,0,1970,1717,1,642,1,9550,2006,1,1,0,540,5,0,1998.0,0,0,0,0,...,0,0,0,0,0,13,0,6,0,2,0,8,0,4,5,2,3,1,5,15,3,0,1,4,1,4,4,2,0,5,1,2,2,3,1,0,2,2,2,1
4,0,0,1,655,350.0,0,12,1053,3,84.0,250000,2000,192,4,8,84,1145,60,1145,9,0,0,2000,2198,1,836,1,14260,2008,1,2,0,490,5,0,2000.0,0,0,0,0,...,0,0,0,0,0,12,0,6,0,4,4,8,2,4,5,2,2,1,5,13,2,0,1,4,1,4,4,2,0,1,1,2,1,0,1,2,2,0,1,3


**L1 feature selection**

In [ ]:
df_X = data_encoded.drop('SalePrice', axis=1)
df_y = data[target]

regressor=LassoCV()
regressor.fit(df_X,df_y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [ ]:
print(regressor.coef_)

[-0.00000000e+00  0.00000000e+00  0.00000000e+00  1.21551932e+01
  3.77567567e+01 -0.00000000e+00  0.00000000e+00  1.89143641e+00
  0.00000000e+00 -0.00000000e+00  3.10718090e+02  2.94345453e+01
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  3.33212256e+01  0.00000000e+00 -0.00000000e+00
 -1.38807469e-01  1.14441742e+02  6.43815898e+01 -0.00000000e+00
  6.80312111e+01  0.00000000e+00  2.87565672e-01 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000

In [ ]:
coef_col = pd.Series(regressor.coef_,index=df_X.columns)
print(coef_col)
print()

sorted_coef = coef_col.sort_values()
print(sorted_coef)

EnclosedPorch    -0.000000
3SsnPorch         0.000000
HalfBath          0.000000
BsmtFinSF1       12.155193
MasVnrArea       37.756757
                   ...    
BsmtFinType1     -0.000000
Condition1        0.000000
HeatingQC        -0.000000
MasVnrType        0.000000
BsmtCond          0.000000
Length: 110, dtype: float64

MiscVal           -0.138807
EnclosedPorch     -0.000000
FireplaceQu       -0.000000
LandSlope          0.000000
Functional         0.000000
                    ...    
MasVnrArea        37.756757
GrLivArea         64.381590
GarageArea        68.031211
YearRemodAdd     114.441742
YearBuilt        310.718090
Length: 110, dtype: float64


In [ ]:
# selected features are the ones with non-zero coefficients
selected_features = sorted_coef[coef_col != 0]
no_of_selected_features = sum(coef_col != 0)
rejected_features = sorted_coef[coef_col == 0]
no_of_rejected_features = sum(coef_col == 0)
total_features = no_of_selected_features + no_of_rejected_features
print("L1 selected only ",no_of_selected_features," features out of ",total_features," from the transformed dataset ")
print(selected_features.index)
print()
print(rejected_features)

L1 selected only  11  features out of  110  from the transformed dataset 
Index(['MiscVal', 'LotArea', '2ndFlrSF', 'BsmtFinSF1', 'WoodDeckSF',
       'TotalBsmtSF', 'MasVnrArea', 'GrLivArea', 'GarageArea', 'YearRemodAdd',
       'YearBuilt'],
      dtype='object')

EnclosedPorch      -0.0
FireplaceQu        -0.0
LandSlope           0.0
Functional          0.0
LotShape           -0.0
                   ... 
MSZoning_RL         0.0
MSZoning_C (all)   -0.0
MSZoning_FV         0.0
LandContour_Bnk    -0.0
MSZoning_RH        -0.0
Length: 99, dtype: float64


**Building and evaluating model**

In [ ]:
df_X_selected_features = df_X[selected_features.index]
df_X_selected_features
# standardize the data
X_array_selected_features = PowerTransformer().fit_transform(df_X_selected_features)
X_array_selected_features

y_array = df_y.to_numpy()

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [ ]:
# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_array_selected_features, y_array, test_size=0.3, random_state=42)

In [ ]:
#build ANN model
model=Sequential()
model.add(Dense(32, input_dim=11, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, input_dim=11, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, verbose=0, epochs=50)

In [ ]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f0a97acb190>>

In [ ]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 2ms/step - loss: 24369010688.0000 - accuracy: 0.0000e+00


[24369010688.0, 0.0]